In [6]:
import bz2
import pickle
from typing import Any
from importlib.metadata import version

In [4]:
def _load_reactions() -> tuple[dict[str, Any], list[Any]]:
    from linex2 import load_data
    from linex2 import utils
    from linex2 import make_all_reaction_list_from_reactome
    from linex2 import make_all_reaction_list_from_rhea
    from linex2 import parse_lipid_reference_table_dict

    # First the reference lipid classes are generated.
    # This is a dictionary with lipid classes as keys, and a "Reference class" reactionect as value
    # We only have reactions available for lipids of these classes.
    reference_lipids = parse_lipid_reference_table_dict(load_data.STANDARD_LIPID_CLASSES)

    # Then we read the reactions we curated from the Rhea database
    # and the reactions from the Reactome database
    # (loading this takes a few seconds)
    rhea_reactions = make_all_reaction_list_from_rhea(
        load_data.RHEA_OTHERS_UNIQUE,
        load_data.RHEA_REACTION_CURATION,
        load_data.RHEA_REACTION_TO_MOLECULE,
        reference_lipids,
        load_data.RHEA_MAPPING,
        verbose=False
    )
    reactome_reactions = make_all_reaction_list_from_reactome(
        load_data.REACTOME_OTHERS_UNIQUE,
        load_data.REACTOME_REACTION_CURATION,
        load_data.REACTOME_REACTION_TO_MOLECULE,
        reference_lipids,
        load_data.REACTOME_REACTION_DETAILS,
        verbose=False
    )

    # Then we combine both reaction lists
    # This is the list (together with the reference lipids you will be working with)
    combined_reactions = utils.combine_reactions(reactome_reactions[0], rhea_reactions[0])

    return reference_lipids, combined_reactions


In [7]:
current_linex_version = version("linex2")
reference_lipids, combined_reactions = _load_reactions()
linex_data = (current_linex_version, reference_lipids, combined_reactions)

In [8]:
with bz2.BZ2File('../data/linex/linex_data.pbz2', "w") as f:
    pickle.dump(linex_data, f)